# Construction and meshing of fracture networks

In this tutorial, we will show:

1. How to define fractures and a fracture network in a 3d domain.
2. How to construct a family of meshes that represent the 3d domain, the fractures and their intersections.
3. Assembly of the grids into a `GridBucket` container that stores all grids, and the geometric relation between them.

Together, these are the first steps towards creating a simulation model for a mixed-dimensional problem in fractured domains.


## Summary
For most simulation purposes, the final grid bucket is all that is needed. Therefore, we start by showing a shortcut for obtaining a `GridBucket` given a set of fractures, a domain and two mesh size parameters. All these will be described in more detail below.
We also mention that access to methods generating grid buckets for a small selection of geometries is available under `pp.grids.standard_grids`.

### Meshing of 2d fractures
Fracture networks in 2d are formed by a set of points, together with their connections. We define this by

In [1]:
import numpy as np
import porepy as pp

# Point coordinates, as a 2xn array
p = np.array([[0, 2, 1, 1], [0, 0, 0, 1]])
# Point connections as a 2 x num_frac arary
e = np.array([[0, 2], [1, 3]])

# The domain contains two fractures: The first from (0, 0) - (0, 2), the second (1, 0) to (1, 1)
# Set domain boundaries
domain = {'xmin': -2, 'xmax': 3, 'ymin': -2, 'ymax': 3}

# Define a fracture network in 2d
network_2d = pp.FractureNetwork2d(p, e, domain)

# Set preferred mesh size close to the fracture, and at the boundary (essentially this is a far-field value)
mesh_args = {'mesh_size_frac': 0.2, 'mesh_size_bound': 0.3}

# Generate a mixed-dimensional mesh
gb = network_2d.mesh(mesh_args)


### 3d meshing
Fractures in 3d are polygons instead of lines. This makes computations of the geometry quite a bit more difficult, and requires further data structure to store the fractures:

In [2]:
# The fractures are specified by their vertices, stored in a numpy array
f_1 = pp.Fracture(np.array([[0, 1, 2, 0], [0, 0, 1, 1], [0, 0, 1, 1]]))
f_2 = pp.Fracture(np.array([[0.5, 0.5, 0.5, 0.5], [-1, 2, 2, -1], [-1, -1, 2, 2]]))

# Also define the domain
domain = {'xmin': -2, 'xmax': 3, 'ymin': -2, 'ymax': 3, 'zmin': -3, 'zmax': 3}

# Define a 3d FractureNetwork, similar to the 2d one
network = pp.FractureNetwork3d([f_1, f_2], domain=domain)
mesh_args = {'mesh_size_frac': 0.3, 'mesh_size_min': 0.2}

# Generate the mixed-dimensional mesh
gb = network.mesh(mesh_args, ensure_matching_face_cell=False)

Note the option ensure_matching_face_cell=False - this is needed to circumvent some issues with the way gmsh produces grids. As far as we are aware, it has no consequences for the simulation quality, but we are working on fixing it.
 
## Import filters 
FractureNetworks (2d and 3d) can also be defined directly from files storing their data, see pp.fracture_importer for details. 
 


# Technical details

We focus on the the technicalities related to 3d meshing; in 2d, meshing is relatively simple (still difficult, but 3d is much worse). 

Functionality for fractures and their intersection are provided in the subpackage `porepy.fracs`. Fractures are defined either as Elliptic fractures, or as convex, planar polygons.

In [3]:
# Specify a fracture by its vertexes, as a 3xn array
pts_1 = np.array([[0, 1, 2, 0], [0, 0, 1, 1], [0, 0, 1, 1]])
f_1 = pp.Fracture(pts_1)

# .. and another fracture, intersecting the first
pts_2 = np.array([[0.5, 0.5, 0.5, 0.5], [-1, 2, 2, -1], [-1, -1, 2, 2]])
f_2 = pp.Fracture(pts_2)
               

We can also specify the fracture as an ellipsis, approximated as a polygon.

In [4]:
# Specify the fracture center
center = np.array([0.1, 0.3, 0.2])
# The minor and major axis
major_axis = 1.5
minor_axis = 0.5

# Rotate the major axis around the center.
# Note that the angle is measured in radians
major_axis_angle = np.pi/6

# So far, the fracture is located in the xy-plane. To define the incline, specify the strike angle, and the dip angle.
# Note that the dip rotation is carried out after the major_axis rotation (recall rotations are non-commutative).
strike_angle = -np.pi/3
dip_angle = -np.pi/3

# Finally, the number of points used to approximate the ellipsis. 
# This is the only optional parameter; if not specified, 16 points will be used.
num_pt = 12
f_3 = pp.EllipticFracture(center, major_axis, minor_axis, major_axis_angle, strike_angle, dip_angle, num_points=num_pt)

The fractures can be joined into a `FractureNetwork`.

In [5]:
network = pp.FractureNetwork3d([f_1, f_2, f_3])

The `FractureNetwork` class is the base for analysis and manipulation of fracture networks. The functionality is expanding on demand. For the moment, the most interesting feature is the export of the fracture network to ParaView (requires the vtk extension of python installed, see installation instruction):

In [6]:
network.to_file('fracture_network.vtu')

The resulting file can be opened in ParaView.

We have not yet set a boundary for the `FractureNetwork`, and effectively for the domain. The boundary is defined as a box, and is imposed in the following way

In [7]:
# The domain is a dictionary with fields xmin, xmax, etc.
domain = {'xmin': -2, 'xmax': 3, 'ymin': -2, 'ymax': 3, 'zmin': -3, 'zmax': 3}
network.impose_external_boundary(domain)

array([0, 1, 2])

Above, we defined the bounding box to not intersect with the fractures. If the domain would have been smaller, fractures that intersect a face of the box would by default (can be overruled) have been truncated so that they are confined within the bounding box.

# Meshing

Our aim is to create a computational mesh that conforms to the fractures, as well as to their intersections (1d lines, 0d points). For the actual grid construction we rely on Gmsh. However, these packages all require that the geometric constraints, that is the fractures, are described as *non-intersecting* polygons [if you know of packages that do not require this, please let us know]. It only takes some thinking to understand why the meshing software would not like to do this themselves; this is a highly challenging task.

PorePy provides functionality for finding intersections between fractures, and splitting them into non-intersecting polygons. Intersections are found by 

In [8]:
network.split_intersections()

### Geometric tolerances and stability of meshing algorithm
A critical concept in meshing of fractured domains is the concept of geometric tolerance: Since we are operating in finite precision aritmethics, two points may or may not be consider equal (or similarly, two lines / planes may or may not intersect), depending on how accurately we consider their representation. At least three concepts come into play here

1. The accuracy of the numerical representation of the objects (accumulated effect of finite precision rounding errors).
2. The accuracy in geological interpretation of fracture data: If the fracture network originates from an interpretation of satellite images, differences measured in centimeters should be treated with some caution
3. The resolution of the computational grid: If points with a certain distance are considered non-equal, this may also require that we resolve their difference in the mesh. In addition, the mesh generator will use its own concept of geometric tolerance for internal calculations.

In PorePy, these issues are attempted resolved as follows: The `FractureNetwork` has an attribute `tol` that represent the geometric tolerance used in the calculation of intersections and subsequent splitting of the fractures. If meshing is done with gmsh, the tolerances used in PorePy and gmsh are related. The approach works reasonably well, but for complex configurations of fracture intersections, stability issues can arise. We are working to iprove these matters.

## Interaction with gmsh

Next, create grids for the domain, as well as for fractures and fracture intersections. This involves creating a config file for the mesh generator that contains geometry description, including fracture planes and their intersections. The mesh is then created by calling gmsh (NOTE: The path to the gmsh executable should be specified in a PorePy config file, type 'porepy.utils.read_config?' for more information). The resuling mesh information is read back to python, and `Grid` objects representing the matrix, fractures and fracture intersections are created.



Gmsh is quite flexible in terms of letting the user set / guide the preferred mesh size in different parts of the domain. PorePy tries to adjust to this adapting the specified mesh size to the geometry. From the user side, two parameters must be specified: mesh_size_frac gives the target mesh size in the absence of geometric constraints, while mesh_size_min gives the minimal mesh size to be specified to Gmsh. What actually happens with the mesh, that is, how Gmsh translates these preferred options into a grid, is another matter. It may take some practice to get this to work properly.


In [9]:
mesh_size_frac = 0.3
mesh_size_min = 0.2
mesh_args = {'mesh_size_frac': mesh_size_frac, 'mesh_size_min': mesh_size_min}

# With the mesh size parameters, we can simply ask the GridBucket to mesh itself:
gb = network.mesh(mesh_args)

# Report the number of cells, faces and nodes before and after.
g = [g for g, _ in gb if g.dim == gb.dim_max()][0]
print('Number of cells: ' + str(g.num_cells))
print('Number of faces: ' + str(g.num_faces))
print('Number of nodes: ' + str(g.num_nodes))


Number of cells: 28982
Number of faces: 60844
Number of nodes: 6110


The GridBucket is in effect a mixed-dimensional mesh that can be used for discretization and visualization.

# Visualization of the mixed-dimensional mesh
The set of meshes in the `GridBucket` can be dumped to ParaView by simply writing

In [10]:
e = pp.Exporter(gb, 'grid_bucket')
e.write_vtu()

/home/eke001/Dropbox/workspace/python/ppdir/src/porepy/viz/exporter.py:1012: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  angle = np.arccos(np.dot(normals[:, fi], reference))


Again, some manipulations in ParaView show how the grids on fracture surfaces intersects with the matrix grid.

<img src='mixed_dimensional_grid.png'  width=200>

